In [2]:
import math
import time
from itertools import product
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pyperclip
from IPython.display import clear_output
from PIL import Image, ImageDraw

import image_color_classics as classics

In [3]:
wheel_real_size = 0.58
hook_real_size = 6 * (10**-3)
wheel_pixel_size = 2400
hook_pixel_size = None  # (hook_real_size / wheel_real_size) * wheel_pixel_size
n_hooks = 180

hooks = classics.generate_hooks(n_hooks, wheel_pixel_size)
through_pixels_dict = classics.build_through_pixels_dict(hooks, n_hooks)


In [ ]:
path = str(Path.cwd() / "images")
assert Path(path).exists()

image_BLACK = classics.prepare_image(path + "/bowie_black_v2.jpg", wheel_pixel_size)
image_wpos = classics.prepare_image(path + "/bowie_black_w.jpg", wheel_pixel_size, weighting=True)
image_wneg = classics.prepare_image(path + "/bowie_black_wneg.jpg", wheel_pixel_size, weighting=True)

image_RED = classics.prepare_image(path + "/bowie_red.jpg", wheel_pixel_size)
image_BLUE = classics.prepare_image(path + "/bowie_blue.jpg", wheel_pixel_size)

image_list = [image_BLACK, image_wpos, image_wneg, image_RED, image_BLUE]
classics.display_images(image_list)


In [ ]:
lines_BLACK = classics.find_lines(
    image_BLACK,
    through_pixels_dict,
    n_lines=1350,
    # ! ^the one I drew used 850 - that's a good number for a very thick pen, but thinner can get away with more
    darkness=200,
    lightness_penalty=0.45,
    w_pos=image_wpos,
    w_neg=image_wneg,
    line_norm_mode="weighted length",
    time_saver=0.5,
    flip_parity=False,
    n_hooks=n_hooks,
    wheel_pixel_size=wheel_pixel_size,
)

lines_RED = classics.find_lines(
    image_RED,
    through_pixels_dict,
    n_lines=250,  # ! 700,
    darkness=240,
    lightness_penalty=0.4,
    line_norm_mode="length",
    time_saver=0.5,
    flip_parity=False,
    n_hooks=n_hooks,
    wheel_pixel_size=wheel_pixel_size,
)

lines_BLUE = classics.find_lines(
    image_BLUE,
    through_pixels_dict,
    n_lines=120,  # ! 500,
    darkness=200,
    lightness_penalty=0.15,
    line_norm_mode="length",
    time_saver=0.5,
    flip_parity=False,
    n_hooks=n_hooks,
    wheel_pixel_size=wheel_pixel_size,
)

In [7]:
all_lines = {
    "black": lines_BLACK,
    "red": lines_RED,
    "blue": lines_BLUE,
}
np.savez("outputs_drawing/bowie_lines_01.npz", **all_lines)

In [10]:
# classics.save_plot([lines_BLUE], [(0, 100, 255)], "david bowie blue output (small)", 4000)
# classics.save_plot(
#     [lines_BLUE, lines_RED],
#     [(0, 100, 255), (255, 0, 0)],
#     "david bowie blue and red output (small)",
#     size=4000,
#     n_hooks=n_hooks,
# )
# classics.save_plot_progress(
#     [lines_BLUE, lines_RED, lines_BLACK],
#     [(0, 100, 255), (255, 0, 0), (0, 0, 0)],
#     "david bowie output (small)",
#     size=4000,
#     n_hooks=n_hooks,
#     proportion_list=[0.2, 0.4, 0.6, 0.8, 1],
# )
classics.save_plot(
    [lines_BLUE, lines_RED, lines_BLACK],
    [(0, 100, 255), (255, 0, 0), (0, 0, 0)],
    "outputs_drawing/bowie_lines_01",
    size=2000,
    n_hooks=n_hooks,
)


In [48]:
gcode_RED = classics.generate_gcode(lines_RED[1:], n_hooks * 2, 0, 0, 250, 250, speed=9000)
gcode_BLUE = classics.generate_gcode(lines_BLUE[1:], n_hooks * 2, 0, 0, 250, 250, speed=9000)
gcode_BLACK = classics.generate_gcode(lines_BLACK[1:], n_hooks * 2, 0, 0, 250, 250, speed=9000)

# pyperclip.copy("\n".join(gcode_RED))
# pyperclip.copy("\n".join(gcode_BLUE))
# pyperclip.copy("\n".join(gcode_BLACK))